<a href="https://colab.research.google.com/github/pablna03/Entrega-EAL/blob/main/AssistEcuaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install bardapi
from IPython.display import clear_output
clear_output()
from bardapi import Bard
import os
os.environ['_BARD_API_KEY'] = 'dQh73eHMJqf8A-MbbNHdpaQzPZiPbadRC0hj8abpxsW8ii715f4u6BINjQbKDGicNi5cZA.'
import sympy as sp
import numpy
from sympy.abc import x
from sympy.polys.polytools import nroots
import math
import cmath
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [88]:
def obtenerOpcion(minimo, maximo, msg):
  opcion = minimo - 1
  while(opcion == minimo - 1):
    try:
        opcion = int(input(msg))
        if opcion < minimo or opcion > maximo:
          print("Opción no válida.")
          opcion = minimo - 1
    except:
        print("Opción no válida.")

  return opcion

In [89]:
def solucion_lineal(coeficientes):
  return [-coeficientes[1] / coeficientes[0]]

In [90]:
def solucion_cuadratica(coeficientes):
  a = coeficientes[0]
  b = coeficientes[1]
  c = coeficientes[2]

  discriminante = b ** 2 - 4 * a * c

  if discriminante < 0:
    real = -b / (2 * a)
    imaginaria = math.sqrt(-discriminante) / (2 * a)
    return [complex(real, imaginaria), complex(real, -imaginaria)]
  else:
    return [(-b + math.sqrt(discriminante)) / (2 * a), (-b - math.sqrt(discriminante)) / (2 * a)]

In [91]:
def metodo_cardano(coeficientes):
  a = coeficientes[1] / coeficientes[0]
  b = coeficientes[2] / coeficientes[0]
  c = coeficientes[3] / coeficientes[0]

  p = (3 * b - a ** 2) / 3
  q = (2 * a ** 3 - 9 * a * b + 27 * c) / 27

  discriminante = (q / 2) ** 2 + (p / 3) ** 3

  tol = 1e-10

  if discriminante > tol:
    u_cub = - q / 2 + math.sqrt(discriminante)
    u = abs(u_cub) ** (1 / 3)
    if u_cub < 0:
      u = -u

    v_cub = - q / 2 - math.sqrt(discriminante)
    v = abs(v_cub) ** (1 / 3)
    if v_cub < 0:
      v = -v

    real = - (u + v) / 2 - a / 3
    imaginaria = (math.sqrt(3) / 2) * (u - v)

    return [u + v - a / 3, complex(real, imaginaria), complex(real, -imaginaria)]

  elif discriminante < -tol:
    theta = math.acos((- q / 2) / (math.sqrt(-((p / 3) ** 3))))
    x = 2 * math.sqrt(-(p / 3))

    return [x * math.cos(theta / 3) - a / 3, x * math.cos((theta + 2 * math.pi) / 3) - a / 3, x * math.cos((theta + 4 * math.pi) / 3) - a / 3]

  else:
    if p == 0 and q == 0:
      return [-a / 3, -a / 3, -a / 3]
    else:
      return [- (3 * q) / (2 * p) - a / 3, - (3 * q) / (2 * p) - a / 3, - (4 * (p ** 2)) / (9 * q) - a / 3]

In [92]:
def metodo_ferrari(coeficientes):
  a = coeficientes[1] / coeficientes[0]
  b = coeficientes[2] / coeficientes[0]
  c = coeficientes[3] / coeficientes[0]
  d = coeficientes[4] / coeficientes[0]

  p = (8 * b - 3 * a ** 2) / 8
  q = (8 * c - 4 * a * b + a ** 3) / 8
  r = (256 * d - 64 * a * c + 16 * (a ** 2) * b - 3 * (a ** 4)) / 256

  tol = 1e-10

  if abs(q) > tol:
    P = metodo_cardano([1, - p / 2, -r, (4 * p * r - q ** 2) / 8])[0]
    R = cmath.sqrt(P ** 2 - r)
    Q = - q / (2 * R)

    sq1 = cmath.sqrt(Q ** 2 - 4 * (P - R))
    sq2 = cmath.sqrt(Q ** 2 - 4 * (P + R))

    roots = [(Q + sq1) / 2 - a / 4, (Q - sq1) / 2 - a / 4, (-Q + sq2) / 2 - a / 4, (-Q - sq2) / 2 - a / 4]
    roots = [x.real if x.imag == 0 else x for x in roots]

    return roots

  else:
    t2 = solucion_cuadratica([1, p, r])

    sq_t1 = cmath.sqrt(t2[0])
    sq_t2 = cmath.sqrt(t2[1])

    roots = [sq_t1 - a / 4, -sq_t1 - a / 4, sq_t2 - a / 4, -sq_t2 - a / 4]
    roots = [x.real if x.imag == 0 else x for x in roots]

    return roots

In [93]:
def aproximacion_numerica(grado, coeficientes):
  if grado == 5:
    p = sp.Poly(coeficientes[0]*x**5 + coeficientes[1]*x**4 + coeficientes[2]*x**3 + coeficientes[3]*x**2 + coeficientes[4]*x + coeficientes[5])
  elif grado == 6:
    p = sp.Poly(coeficientes[0]*x**6 + coeficientes[1]*x**5 + coeficientes[2]*x**4 + coeficientes[3]*x**3 + coeficientes[4]*x**2 + coeficientes[5]*x + coeficientes[6])
  elif grado == 7:
    p = sp.Poly(coeficientes[0]*x**7 + coeficientes[1]*x**6 + coeficientes[2]*x**5 + coeficientes[3]*x**4 + coeficientes[4]*x**3 + coeficientes[5]*x**2 + coeficientes[6]*x + coeficientes[7])
  elif grado == 8:
    p = sp.Poly(coeficientes[0]*x**8 + coeficientes[1]*x**7 + coeficientes[2]*x**6 + coeficientes[3]*x**5 + coeficientes[4]*x**4 + coeficientes[5]*x**3 + coeficientes[6]*x**2 + coeficientes[7]*x + coeficientes[8])
  elif grado == 9:
    p = sp.Poly(coeficientes[0]*x**9 + coeficientes[1]*x**8 + coeficientes[2]*x**7 + coeficientes[3]*x**6 + coeficientes[4]*x**5 + coeficientes[5]*x**4 + coeficientes[6]*x**3 + coeficientes[7]*x**2 + + coeficientes[8]*x + coeficientes[9])
  else:
    p = sp.Poly(coeficientes[0]*x**10 + coeficientes[1]*x**9 + coeficientes[2]*x**8 + coeficientes[3]*x**7 + coeficientes[4]*x**6 + coeficientes[5]*x**5 + coeficientes[6]*x**4 + coeficientes[7]*x**3 + coeficientes[8]*x**2 + coeficientes[9]*x + coeficientes[10])

  roots = nroots(p, n=15, maxsteps=50, cleanup=True)

  return roots


In [94]:
def obtenerEcuacion(coeficientes, grado):
  coeficientes_grados = [(coeficientes[x], grado - x) for x in range(grado + 1)]
  ecuacion = ""
  for coeficiente, g in coeficientes_grados:
    if g == 0:
      if coeficiente == 0:
        ecuacion += " = 0"
      else:
        ecuacion += " + {} = 0".format(coeficiente)
    elif coeficiente != 0:
      nuevo_coeficiente = "" if coeficiente == 1 else coeficiente
      if g == grado and g == 1:
        ecuacion += "{}x".format(nuevo_coeficiente, g)
      elif g == grado:
        ecuacion += "{}x^{}".format(nuevo_coeficiente, g)
      elif g == 1:
        ecuacion += " + {}x".format(nuevo_coeficiente)
      else:
        ecuacion += " + {}x^{}".format(nuevo_coeficiente, g)

  return ecuacion

def obtenerCoeficientes(grado):
  letras = [chr(97 + x) for x in range(grado + 1)]
  ecuacion = obtenerEcuacion(letras, grado)
  print("Introduzca los coeficientes para la ecuación", ecuacion)
  coeficientes = []
  a = 0
  while(a == 0):
    coeficientes = []
    for letra in letras:
      coeficientes.append(obtenerOpcion(float('-inf'), float('inf'), "{} = ".format(letra)))
    a = coeficientes[0]
    if a == 0:
      print("El coeficiente a debe ser distinto que 0")

  return coeficientes

def resolver_ecuacion(grado, coeficientes):
  if grado == 1:
    return solucion_lineal(coeficientes)
  elif grado == 2:
    return solucion_cuadratica(coeficientes)
  elif grado == 3:
    return metodo_cardano(coeficientes)
  elif grado == 4:
    return metodo_ferrari(coeficientes)
  else:
    return aproximacion_numerica(grado, coeficientes)

def obtenerSoluciones():
  clear_output()
  grado = obtenerOpcion(0, 10, "Introduzca el grado de la ecuación que desea resolver ([1...10] | [0] para volver a la pantalla anterior): ")

  if grado != 0:
    coeficientes = obtenerCoeficientes(grado)
    soluciones = resolver_ecuacion(grado, coeficientes)

    print("Las soluciones de la ecuación", obtenerEcuacion(coeficientes, grado), "son:")
    for index, solucion in enumerate(soluciones):
      print("\tx{} = {}".format(index, solucion))

In [95]:
def obtenerInformacionHistoria():
  clear_output()
  print("¿De las ecuaciones de qué grado deseas conocer un poco sobre la historia?")
  print("\t1. Ecuaciones lineales (Primer grado)")
  print("\t2. Ecuaciones cuadráticas (Segundo grado)")
  print("\t3. Ecuaciones cúbicas (Tercer grado)")
  print("\t4. Ecuaciones cuárticas (Cuarto grado)")
  print("\t5. Ecuaciones de grado superior")
  print("\t0. Volver a la pantalla anterior")

  opcion = obtenerOpcion(0, 5, "Opción: ")
  if opcion != 0:
    if opcion < 5:
      print(Bard().get_answer("Cuéntame un poco sobre la historia de la resolución por radicales de la ecuación de grado {}. No incluyas la fórmula. (Máximo 200 palabras)".format(opcion))['content'])
    else:
      print(Bard().get_answer("Cuéntame un poco sobre la historia de la demostración de que no existe una solución por radicales de las ecuaciones de grado mayor o igual que 5. (Máximo 200 palabras)")['content'])


In [96]:
def mostrarFormulasRadicales():
  clear_output()
  print("¿De las ecuaciones de qué grado deseas conocer la fórmula?")
  print("\t1. Ecuaciones lineales (Primer grado)")
  print("\t2. Ecuaciones cuadráticas (Segundo grado)")
  print("\t3. Ecuaciones cúbicas (Tercer grado)")
  print("\t4. Ecuaciones cuárticas (Cuarto grado)")
  print("\t0. Volver a la pantalla anterior")

  opcion = obtenerOpcion(0, 4, "Opción: ")
  if opcion != 0:
    print("La fórmula para resolver ecuaciones de grado {} es:\n".format(opcion))
    ruta_imagen = '/content/Formula_ecuacion_grado_{}.png'.format(opcion)
    imagen = mpimg.imread(ruta_imagen)
    plt.imshow(imagen)
    plt.axis('off')
    plt.show()
    plt.close()
    return

In [98]:
def assistEcuaciones():

  opcion = 1
  while(opcion != 0):
    print("¡Bienvenido a AssistEcuaciones: Plataforma de Resolución Algebraica!\n")
    print(" ¿Que desea hacer?")
    print("\t1. Obtener las soluciones de una ecuación")
    print("\t2. Conocer información sobre la historia de la resolución de las ecuaciones")
    print("\t3. Conocer las fórmulas de la resolución por radicales de ecuaciones de hasta cuarto grado")
    print("\t0. Salir")

    opcion = obtenerOpcion(0, 3, "Opción: ")
    if opcion == 1:
      obtenerSoluciones()
    elif opcion == 2:
      obtenerInformacionHistoria()
    elif opcion == 3:
      mostrarFormulasRadicales()

    if opcion != 0:
      input("Pulse Intro para continuar...")
      clear_output()

  print("¡Hasta la próxima!")

In [99]:
assistEcuaciones()

¡Bienvenido a AssistEcuaciones: Plataforma de Resolución Algebraica!

 ¿Que desea hacer?
	1. Obtener las soluciones de una ecuación
	2. Conocer información sobre la historia de la resolución de las ecuaciones
	3. Conocer las fórmulas de la resolución por radicales de ecuaciones de hasta cuarto grado
	0. Salir
Opción: 0
¡Hasta la próxima!
